In [2]:
from torchmetrics.functional import char_error_rate, word_error_rate
import re
import pandas as pd

In [3]:
context.catalog.list()

['express_expense',
 'labels',
 'homography_points',
 'homography_points_segmentation',
 'homography_points_segmentation_batched_hull',
 'homography_points_segmentation_batched_hough',
 'homography_boxes',
 'homography_boxes_segmentation',
 'homography_boxes_segmentation_batched',
 'mask',
 'masks',
 'masks_batched',
 'image_perturbation_metrics',
 'image_perturbation_report',
 'distorted',
 'distorted_small',
 'express_small',
 'express_test',
 'pipeline_sample_images',
 'methods_original_ocr_text',
 'methods_distorted_ocr_text',
 'ablation_ocr_text',
 'ablation_ocr_distorted_text',
 'parameters']

In [38]:
p = context.catalog.load("ablation_ocr_distorted_text")

[06/14/23 10:11:12] INFO     Loading data from 'ablation_ocr_distorted_text' (JSONDataSet)...   ]8;id=548129;file:///home/zig/.pyenv/versions/3.10.8/envs/sibigrapi23/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=418587;file:///home/zig/.pyenv/versions/3.10.8/envs/sibigrapi23/lib/python3.10/site-packages/kedro/io/data_catalog.py#343\343]8;;\

In [39]:
p.keys()

dict_keys(['original', 'sh', 'mp', 'sm', 'sr', 'hm'])

In [40]:
p[list(p.keys())[0]][0]

{'file_name': '0-1000-receipt.jpg',
 'pipeline_time': 0.06354237500000082,
 'ocr_text': ''}

In [41]:
labels = context.catalog.load("labels")

[06/14/23 10:11:13] INFO     Loading data from 'labels' (PartitionedDataSet)...                 ]8;id=92503;file:///home/zig/.pyenv/versions/3.10.8/envs/sibigrapi23/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=711151;file:///home/zig/.pyenv/versions/3.10.8/envs/sibigrapi23/lib/python3.10/site-packages/kedro/io/data_catalog.py#343\343]8;;\

In [42]:
p.keys()

dict_keys(['original', 'sh', 'mp', 'sm', 'sr', 'hm'])

In [43]:
def _cleanText(t):
    t = re.sub(r" +", " ", t) # replace multiple spaces with one
    t = re.sub(r"[\s\n]+\n", "\n", t).strip() # remove empty lines
    return t


In [48]:
def generate_report(ocr_pipelines, labels):
    stop = False
    pipeline_results = dict()
    for pipeline_name, ocr_results in ocr_pipelines.items():
        results_errors = list()
        results_binary = list()
        for ocr_result in ocr_results:
            try:
                # label = labels[ocr_result["file_name"].replace(".jpg", ".txt")]() #usar essa linha para os datasets não distorcidos
                label = labels[ocr_result["file_name"][2:].replace(".jpg", ".txt")]() #usar essa linha para o dataset distorcido
            except:
                continue
            if (ocr_result["ocr_text"]):
                results_errors.append({
                    'wer': float(word_error_rate(preds=_cleanText(ocr_result["ocr_text"]), target=_cleanText(label))),
                    'cer': float(char_error_rate(preds=_cleanText(ocr_result["ocr_text"]), target=_cleanText(label))),
                })
            results_binary.append({
                'br': 1 if ocr_result["ocr_text"] else 0,
                'ba': 0,
                'run_time': ocr_result["pipeline_time"]
            })
        df_results_errors = pd.DataFrame(results_errors)
        df_results_binary = pd.DataFrame(results_binary)
        try:
            df_results_binary['ba'] = df_results_errors['cer'].le(0.05).astype(int)
        except:
            continue
        df_results = pd.concat([df_results_errors.describe().loc[["mean", "std"], :].apply(lambda x: round(x * 100, 1)), df_results_binary.describe().loc[["mean", "std"], :].apply(lambda x: round(x * 100, 1))], axis=1)
        df_results["run_time"] = (df_results["run_time"] * 10).apply(round) / 1000
        df_results = df_results.T
        pipeline_results[pipeline_name] = df_results["mean"].astype(str) + " (" + df_results["std"].astype(str) + ")"

    return pd.DataFrame(pipeline_results).T

In [49]:
a = generate_report(p, labels)

In [50]:
a

,wer,cer,br,ba,run_time
original,84.2 (21.1),74.3 (27.2),19.0 (39.2),0.0 (0.0),0.056 (0.086)
sh,92.1 (16.5),75.5 (23.3),21.8 (41.3),0.0 (0.0),0.015 (0.029)
mp,93.1 (17.2),76.9 (22.4),23.2 (42.3),0.0 (0.0),0.015 (0.033)
sm,95.7 (17.1),79.5 (20.2),27.9 (44.9),0.0 (0.0),0.012 (0.031)
sr,100.0 (25.7),78.7 (23.2),31.1 (46.3),0.0 (0.0),53.523 (65.735)
hm,98.0 (37.8),70.8 (27.9),67.7 (46.8),0.6 (7.8),53.997 (65.685)
